In [42]:
import os, math
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.datasets import cifar10, mnist
import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization 
from tensorflow.keras.layers import Conv2D, Flatten, Activation
from tensorflow.keras.layers import Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mse

In [2]:
(x_train, _),(x_test, _) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
image_size = x_train[0].shape[1]
image_size

28

In [4]:
gen_resize_img = image_size // 4
gen_kernel_size = 5
gen_layers_filter = [128, 64, 32, 1]

In [15]:
class ConvTransBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides):
        super().__init__()
        self.bn = BatchNormalization()  # check training flag
        self.act = Activation(activation='relu')
        self.conv2D_trans = Conv2DTranspose(filters=filters,
                                kernel_size=kernel_size,
                                strides=strides,
                                padding='same')
        
    def call(self, inputs, training=False):
        x = self.bn(inputs)
        x = self.act(x)
        return self.conv2D_trans(x)


In [22]:
noise = np.random.uniform(0, 1, size=[10000, 100])


In [26]:
model = Sequential()
model.add(Input(noise.shape[1]))
model.add(Dense(14*14*32, activation="relu"))
model.add(Reshape([14, 14, 32]))
model.add(ConvTransBlock(32, 5, 2))
model.add(ConvTransBlock(16, 5, 1))
model.add(Conv2DTranspose(1, 5, padding="same", activation="sigmoid"))


In [ ]:
model.compile(loss="MSE", metrics=["MAE"])
model.fit(noise, x_train.reshape(-1, 28, 28, 1)[:10000])

In [35]:
class Generator(tf.keras.models.Model):
    def __init__(self, filters, kernel_size, resize_img):
        super().__init__()
        self.dense1 = Dense(resize_img * resize_img * filters[0])
        self.reshape = Reshape([resize_img, resize_img, filters[0]])
        self.conv2dtrans = []
        for i, _filter in enumerate(filters):
            if i <= 1:
                strides = 2
            else:
                strides = 1
            self.conv2dtrans.append(ConvTransBlock(_filter, kernel_size, strides))
        
        self.act = Activation("sigmoid")

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.reshape(x)
        for conv in self.conv2dtrans:
            x = conv(x)
        return self.act(x)




In [48]:
gen = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)

In [49]:
gen.compile(loss="mse", metrics=["mae"])

In [52]:
gen.fit(noise, x_train.reshape(-1, 28, 28, 1)[:10000], batch_size=64)

 79/157 [==============>...............] - ETA: 1:15 - loss: 7239.7051 - mae: 33.7319

KeyboardInterrupt: ignored